# A notebook to perform QC on the PIPS compass and wind observations

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.stats.mstats import zscore
from scipy.special import gamma as gammafunc
from scipy import ndimage
from metpy.plots import StationPlot
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
import warnings
warnings.simplefilter('ignore')
%matplotlib widget

In [ ]:
# Read in the original PIPS netcdf files

# PIPS_input_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'
# PIPS_output_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data_for_EOL/'

PIPS_base_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/PIPS_data/'
# PIPS_base_dir = '/Users/dawson29/Dropbox/Projects/PERiLS/obsdata/2023/'

deployment_name = 'IOP2_033022' # 'IOP5_040523' # 'IOP2_030323' # 'IOP5_040523' # 'IOP4_033123' # 'IOP3_032423' # 'IOP3_040522'
PIPS_input_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf')
PIPS_output_dir = os.path.join(PIPS_base_dir, deployment_name, 'netcdf_wind_compass_QC_10s')
if not os.path.exists(PIPS_output_dir):
    os.makedirs(PIPS_output_dir)

# IOP1 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B']
# IOP2 2022
PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS3B']
# IOP3 2022
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
# IOP3 2023
# PIPS_names = ['PIPS2A', 'PIPS3A']
# IOP2, IOP4 or IOP5 2023
# PIPS_names = ['PIPS1A', 'PIPS1B', 'PIPS2A', 'PIPS2B', 'PIPS3A', 'PIPS3B']
parsivel_interval = 10
interval_str = '10S'
parsivel_filenames = ['parsivel_combined_{}_{}_{:d}s.nc'.format(deployment_name, PIPS_name, parsivel_interval)
                      for PIPS_name in PIPS_names]
parsivel_filepaths = [os.path.join(PIPS_input_dir, parsivel_filename) for parsivel_filename in parsivel_filenames]
output_parsivel_filepaths = [os.path.join(PIPS_output_dir, parsivel_filename) 
                             for parsivel_filename in parsivel_filenames]
conv_filenames = ['conventional_raw_{}_{}.nc'.format(deployment_name, PIPS_name) for PIPS_name in PIPS_names]
conv_filepaths = [os.path.join(PIPS_input_dir, conv_filename) for conv_filename in conv_filenames]
output_conv_filepaths = [os.path.join(PIPS_output_dir, conv_filename) for conv_filename in conv_filenames]
parsivel_ds_dict = {}
conv_ds_dict = {}
for PIPS_name, parsivel_filepath, conv_filepath in zip(PIPS_names, parsivel_filepaths, conv_filepaths):
    try:
        parsivel_ds_dict[PIPS_name] = xr.load_dataset(parsivel_filepath)
    except:
        parsivel_ds_dict[PIPS_name] = None
    conv_ds_dict[PIPS_name] = xr.load_dataset(conv_filepath)

In [ ]:
PIPS_to_check = 'PIPS3B'

In [ ]:
parsivel_ds = parsivel_ds_dict[PIPS_to_check]
parsivel_ds

In [ ]:
conv_ds = conv_ds_dict[PIPS_to_check]
conv_ds

In [ ]:
fig, ax = plt.subplots()
conv_ds.plot.scatter(x='GPS_lon', y='GPS_lat', ax=ax)

In [ ]:
print(conv_ds['GPS_status'][:100])

In [ ]:
# Plot the timeseries of compass headings
fig, ax = plt.subplots()
conv_ds['compass_dir'].plot(ax=ax)
try:
    parsivel_ds['compass_dir'].plot(ax=ax)
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
compass_zscores = zscore(conv_ds['compass_dir'], nan_policy='omit')
print(compass_zscores)
fig, ax = plt.subplots()
compass_zscores.plot(ax=ax, ls='None', marker='o', ms=1.)

In [ ]:
# Set to nan any compass headings more than x standard deviation from the mean based on above plot
cleaned_compass_dir = conv_ds['compass_dir'].where(np.abs(compass_zscores) < 0.5)
fig, ax = plt.subplots()
cleaned_compass_dir.plot(ax=ax, ls='None', marker='o', ms=1.)

In [ ]:
# Interpolate across NaN's
# TODO: the following doesn't take into account if the compass direction crosses 360 deg. Fix this. For now it is
# ok because in the cases I'm looking at this doesn't happen.
# cleaned_compass_dir = cleaned_compass_dir.interpolate_na(dim='time', use_coordinate=True)
# fig, ax = plt.subplots()
# cleaned_compass_dir.plot(ax=ax, ls='None', marker='o', ms=1.)

In [ ]:
# Take the average of the cleaned compass directions. We'll use that to recompute the wind directions
avg_compass_dir = cleaned_compass_dir.mean(dim='time', skipna=True).item()
print(avg_compass_dir)

In [ ]:
# Plot the original wind directions
fig, ax = plt.subplots()
conv_ds['winddirabs'].plot(ax=ax, ls='None', marker='o', ms=1.)
ax.set_ylim(0., 360.)

In [ ]:
# Recompute wind directions using average of cleaned compass directions
winddirabs = np.mod(avg_compass_dir + conv_ds['winddirrel'], 360.)
fig, ax = plt.subplots()
winddirabs.plot(ax=ax, ls='None', marker='o', ms=1.)
ax.set_ylim(0., 360.)

In [ ]:
conv_ds['compass_dir'] = cleaned_compass_dir
conv_ds['compass_dir'].attrs['average'] = avg_compass_dir
conv_ds['winddirabs'] = winddirabs

In [ ]:
conv_ds

In [ ]:
parsivel_ds

In [ ]:
# Resample the winds to the parsivel times and replace the old ones
try:
    PSD_datetimes = pips.get_PSD_datetimes(parsivel_ds['VD_matrix'])
    sec_offset = PSD_datetimes[0].second
    new_wind_dict = pips.resample_wind_da(conv_ds['winddirabs'], conv_ds['windspd'], interval_str, 
                                          sec_offset, gusts=True, gustintvstr='3S')
except:
    sec_offset = None
    new_wind_dict = None
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
# Plot original resampled wind directions
try:
    fig, ax = plt.subplots()
    parsivel_ds['winddirabs'].plot(ax=ax, ls='None', marker='o', ms=1.)
    ax.set_ylim(0., 360.)
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
# Plot fixed resampled wind directions
try:
    fig, ax = plt.subplots()
    new_wind_dict['winddirabs'].plot(ax=ax, ls='None', marker='o', ms=1.)
    ax.set_ylim(0., 360.)
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
# Plot original resampled compass directions
try:
    fig, ax = plt.subplots()
    parsivel_ds['compass_dir'].plot(ax=ax, ls='None', marker='o', ms=1.)
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
def resample_compass_da(compass_dir, offset, intervalstr):
    offset_str = pips.get_interval_str(offset)
    # Compute x- and y-components of compass direction
    x = np.cos(np.deg2rad(-compass_dir + 270.))
    y = np.sin(np.deg2rad(-compass_dir + 270.))
    
    x_avg = x.resample(time=intervalstr, label='right', closed='right', offset=offset_str).mean()
    y_avg = y.resample(time=intervalstr, label='right', closed='right', offset=offset_str).mean()
    
    # Need to use %360 to keep direction between 0 and 360 degrees
    compass_dir_avg = (270.0 - (180. / np.pi) * np.arctan2(y_avg, x_avg)) % 360.
    return compass_dir_avg



In [ ]:
# Resample compass directions using corrected values
try:
    cleaned_compass_dir_avg = resample_compass_da(cleaned_compass_dir, sec_offset, interval_str)
    fig, ax = plt.subplots()
    cleaned_compass_dir_avg.plot(ax=ax, ls='None', marker='o', ms=1.)
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
# Replace existing resampled wind and compass variables in the parsivel_ds. The times should automatically
# already match up
try:
    for key, val in new_wind_dict.items():
        # print(key+'_corrected')
        # parsivel_ds[key+'_corrected'] = val
        parsivel_ds[key] = val
        
    parsivel_ds['compass_dir'] = cleaned_compass_dir_avg
    avg_compass_dir = cleaned_compass_dir_avg.mean(dim='time', skipna=True).item()
    parsivel_ds['compass_dir'].attrs['average'] = avg_compass_dir
    print(avg_compass_dir)
    all(cleaned_compass_dir_avg['time'] == parsivel_ds['time'])
except:
    print(f"No Parsivel data for {PIPS_to_check}")

In [ ]:
# Now save to new output directory
for PIPS_name, output_parsivel_filepath, output_conv_filepath in zip(PIPS_names, 
                                                                     output_parsivel_filepaths, 
                                                                     output_conv_filepaths):
    if PIPS_name == PIPS_to_check:
        try:
            print("Saving {}".format(output_parsivel_filepath))
            parsivel_ds.to_netcdf(output_parsivel_filepath)
        except:
            print(f"No Parsivel data for {PIPS_to_check}")
        print("Saving {}".format(output_conv_filepath))
        conv_ds.to_netcdf(output_conv_filepath)